In [ ]:
using DelimitedFiles
using Plots
using LaTeXStrings
using Printf

In [ ]:
using Dates

In [ ]:
function time2str(t)
    ms = Millisecond(round(1000 * t))
    dt = convert(DateTime, ms)
    tt = convert(Time, dt)
    return string(tt)
end

---
# CSV

In [ ]:
using CSV

In [ ]:
df = CSV.read("results.txt", delim=" ", ignorerepeated=true)

In [ ]:
df[df.method.=="IAPG",[:n, :γ, :k, :fgs, :time]]

---
# DataFrames

In [ ]:
using DataFrames

In [ ]:
time2str(sum(df.time))

In [ ]:
describe(df)

In [ ]:
dfgroupbymethod = groupby(df, :method);

In [ ]:
for i in eachindex(dfgroupbymethod)
    display(i.method)
    display(unstack(dfgroupbymethod[i], :n, :γ, :fgs))
end

---
# NamedArrays

In [ ]:
using NamedArrays

In [ ]:
ns = collect(100:100:800)
γs = collect(0.1:0.1:1.0)

for i in eachindex(dfgroupbymethod)
    println(i.method)
    A = Array{Int}(unstack(dfgroupbymethod[i], :n, :γ, :fgs))
    N = NamedArray(A[:,2:end], (ns, γs), ("n", "γ"))
    println(N)
    println()
end

---
# heatmap

In [ ]:
A = Array{Int}(unstack(dfgroupbymethod[(method="IAPG",)], :n, :γ, :fgs))
IAPGfgcalls = NamedArray(A[:,2:end], (ns, γs), ("n", "γ"))

In [ ]:
A = Array{Int}(unstack(dfgroupbymethod[(method="IR",)], :n, :γ, :fgs))
IRfgcalls = NamedArray(A[:,2:end], (ns, γs), ("n", "γ"))

In [ ]:
A = Array{Int}(unstack(dfgroupbymethod[(method="IER",)], :n, :γ, :fgs))
IERfgcalls = NamedArray(A[:,2:end], (ns, γs), ("n", "γ"))

In [ ]:
X = IAPGfgcalls - IERfgcalls

In [ ]:
extrema(X)

In [ ]:
ns = [string(n) for n=100:100:800]
γs = [@sprintf("%.1f", γ) for γ=0.1:0.1:1.0]
plt = heatmap(ns, γs, IAPGfgcalls - IERfgcalls, 
    size=(800, 600),
    #aspect_ratio=1.0,
    xlabel=L"n",
    ylabel=L"\gamma",
    title="Reduction in function evaluations (IER v. IAPG)",
    c=:RdBu,
    clims=(-400, 400),
)

In [ ]:
savefig(plt, "figs/heatmap.png")

In [ ]:
clibraries()

In [ ]:
showlibrary(:colorbrewer)

In [ ]:
for colorlib in clibraries()
    println(colorlib)
    display(showlibrary(colorlib))
end

---
# BenchmarkProfiles

In [ ]:
using BenchmarkProfiles

In [ ]:
T = [IAPGfgcalls.array[:] IRfgcalls.array[:] IERfgcalls.array[:]]
solvernames = ["IAPG", "IR", "IER"]

plt = performance_profile(T, solvernames, logscale=false, 
    size=(800, 600),
    legend=:bottomright, 
    style=:auto,
    linecolor=:black)

In [ ]:
savefig(plt, "figs/performance_profile.pdf")

In [ ]:
T = [IAPGfgcalls.array[:] IRfgcalls.array[:]]
IERfgcallsames = ["IAPG", "IR"]

plt = performance_profile(T, solvernames, logscale=false, 
    size=(800, 600),
    legend=:bottomright, 
    style=:auto,
    linecolor=:black)

In [ ]:
T = [IAPGfgcalls.array[:] IERfgcalls.array[:]]
solvernames = ["IAPG", "IER"]

plt = performance_profile(T, solvernames, logscale=false, 
    size=(800, 600),
    legend=:bottomright, 
    style=:auto,
    linecolor=:black)

In [ ]:
T = [IRfgcalls.array[:] IERfgcalls.array[:]]
solvernames = ["IR", "IER"]

plt = performance_profile(T, solvernames, logscale=false, 
    size=(800, 600),
    legend=:bottomright, 
    style=:auto,
    linecolor=:black)